# YOLO - Inferencia
## Detección de objetos

**Autor original:** @theAIGuysCode<br>
**Adaptado:** @maxiyommi<br>
**Fecha:** jun 2021<br>
**Descripción:** Notebook para inferir resultados.br>

## Preparación del entorno

* Crear entorno virtual (solo la primera vez).

``` bash
    python -m venv yolo_inference # crea el entorno virtual 
```

* Activar entorno virtual (siempre).

``` bash
    source ml/bin/activate # activa el entorno virtual
    deactivate # desactiva el entorno virtual
    rm -r env # eliminar el entorno virtual
```

* Instalar librerias en el entorno virtual creado (solo la primera vez, a menos que se actualice).

``` bash
    pip3 install -r requirements.txt 
```

## Clonación y construcción de Darknet (por única vez)

In [1]:
# clonar repositorio darknet
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15097, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 15097 (delta 8), reused 9 (delta 2), pack-reused 15072
Receiving objects: 100% (15097/15097), 13.48 MiB | 3.75 MiB/s, done.
Resolving deltas: 100% (10253/10253), done.


### Inferencia con uso de GPU (opcional)

In [27]:
# cambie el makefile para tener GPU y OPENCV habilitados
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=1/GPU=0/' Makefile
!sed -i 's/CUDNN=1/CUDNN=0/' Makefile
!sed -i 's/CUDNN_HALF=1/CUDNN_HALF=0/' Makefile

In [ ]:
# Verificar la existencia de CUDA
!/usr/local/cuda/bin/nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0


In [3]:
%cd darknet
# make darknet (crea darknet para que luego pueda usar el archivo ejecutable darknet para ejecutar o entrenar detectores de objetos)
!make

/home/max/Dropbox/Git/proyectos/ml_auli/ml_uali/notebooks/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -c ./src/image_opencv.cpp -o obj/image_opencv.o
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -c ./src/http_stream.cpp -o obj/http_stream.o
./src/http_stream.cpp: In member function ‘bool JSON_sender::write(const char*)’:
./src/http_stream.cpp:253:21: warning: unused variable ‘n’ [-Wunused-variable]
  253 |                 int n = _write(client, outputbuf, outlen);
      |                     ^
./src/http_stream.cpp: In function ‘void set_track_id(detection*, int, float, float, float, int, int, int)’:
./src/http_stream.cpp:867:27: warning: comparison of integer expressions of different signedness: ‘int’ and ‘std::vector<detection_t>::size_type’ {aka ‘long un

## Descargando pesos de YOLOv4

### Entrenamiento propio

yolov4-obj_best.weights URL: https://drive.google.com/file/d/1-5eprW8D2Si3gZOqaN4QadHOFhvu6OWT/view?usp=sharing

yolov4-obj_final.weights URL: https://drive.google.com/file/d/1-00dw9YYg9uz1qFGxxJbmjSC_7C6yG_H/view?usp=sharing

In [4]:
# yolov4-obj_best.weights
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1-00dw9YYg9uz1qFGxxJbmjSC_7C6yG_H' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1-00dw9YYg9uz1qFGxxJbmjSC_7C6yG_H" -O ../yolov4-obj_final.weights && rm -rf /tmp/cookies.txt

/home/max/Dropbox/Git/proyectos/ml_auli/ml_uali/notebooks
--2021-06-14 21:33:38--  https://docs.google.com/uc?export=download&confirm=RL64&id=1-00dw9YYg9uz1qFGxxJbmjSC_7C6yG_H
Resolving docs.google.com (docs.google.com)... 2800:3f0:4002:80b::200e, 172.217.162.14
Connecting to docs.google.com (docs.google.com)|2800:3f0:4002:80b::200e|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0k-5s-docs.googleusercontent.com/docs/securesc/fb5smph1kr8sf0r4483licdb8hko0pkm/hminqlkblftnnqfcdkgjproi1nu6n3op/1623717150000/04394533313268812415/11247419704206750502Z/1-00dw9YYg9uz1qFGxxJbmjSC_7C6yG_H?e=download [following]
--2021-06-14 21:33:38--  https://doc-0k-5s-docs.googleusercontent.com/docs/securesc/fb5smph1kr8sf0r4483licdb8hko0pkm/hminqlkblftnnqfcdkgjproi1nu6n3op/1623717150000/04394533313268812415/11247419704206750502Z/1-00dw9YYg9uz1qFGxxJbmjSC_7C6yG_H?e=download
Resolving doc-0k-5s-docs.googleusercontent.com (doc-0k-5s-docs.googleusercontent.co

### Descargando data_training

In [13]:
# descargar comprimido de data_training
!wget -O ../data_training.zip https://github.com/githubuali/ml_uali/blob/main/training/yolo_v1/data_training.zip?raw=true

# descomprima los conjuntos de datos 
!unzip ../data_training.zip -d ../data_training

# copiar obj.name
!cp ../data_training/obj.names data

# eliminando carpetas
!rm -rf ../data_training.zip

--2021-06-14 22:08:26--  https://github.com/githubuali/ml_uali/blob/main/training/yolo_v1/data_training.zip?raw=true
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/githubuali/ml_uali/raw/main/training/yolo_v1/data_training.zip [following]
--2021-06-14 22:08:27--  https://github.com/githubuali/ml_uali/raw/main/training/yolo_v1/data_training.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/githubuali/ml_uali/main/training/yolo_v1/data_training.zip [following]
--2021-06-14 22:08:27--  https://raw.githubusercontent.com/githubuali/ml_uali/main/training/yolo_v1/data_training.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githu

### Ejecutando Darknet y YOLOv4 (con imagenes por URL)

URL: https://drive.google.com/file/d/1aLTw006n7dTYk_9P8TsY7tN-3xd-txLW/view?usp=sharing

In [14]:
# descargar comprimido de imagenes de prueba
!wget -O ../image_ypf.zip https://github.com/githubuali/ml_uali/blob/main/data/image_ypf.zip?raw=true

# descomprima los conjuntos de datos 
!unzip ../image_ypf.zip

# eliminando carpetas
!rm -rf ../image_ypf.zip

--2021-06-14 22:08:39--  https://github.com/githubuali/ml_uali/blob/main/test_image/image_ypf.zip?raw=true
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/githubuali/ml_uali/raw/main/test_image/image_ypf.zip [following]
--2021-06-14 22:08:40--  https://github.com/githubuali/ml_uali/raw/main/test_image/image_ypf.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/githubuali/ml_uali/main/test_image/image_ypf.zip [following]
--2021-06-14 22:08:40--  https://raw.githubusercontent.com/githubuali/ml_uali/main/test_image/image_ypf.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connecte

In [15]:
# Necesitamos configurar nuestro cfg personalizado en modo de prueba (unica vez)
!sed -i 's/batch=64/batch=1/' ../data_training/yolov4-obj.cfg
!sed -i 's/subdivisions=16/subdivisions=1/' ../data_training/yolov4-obj.cfg

/home/max/Dropbox/Git/proyectos/ml_auli/ml_uali/notebooks/data_training


In [ ]:
https://ellibrodepython.com/tiempo-ejecucion-python

In [3]:
#ejecute su detector personalizado con este comando (cargue una imagen en su unidad de Google para probar, la bandera thresh establece la precisión que debe tener la detección para mostrarla)
!./darknet detector test ../data_training/obj.data ../data_training/yolov4-obj.cfg ../yolov4-obj_final.weights -ext_output -dont_show -out result.json < ../eventos/S-N-20210630/images.txt 

 GPU isn't used 
 OpenCV isn't used - data augmentation will be slow 
mini_batch = 1, batch = 1, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 conv     32       3 x 3/ 1    608 x 608 x   3 ->  608 x 608 x  32 0.639 BF
   1 conv     64       3 x 3/ 2    608 x 608 x  32 ->  304 x 304 x  64 3.407 BF
   2 conv     64       1 x 1/ 1    304 x 304 x  64 ->  304 x 304 x  64 0.757 BF
   3 route  1 		                           ->  304 x 304 x  64 
   4 conv     64       1 x 1/ 1    304 x 304 x  64 ->  304 x 304 x  64 0.757 BF
   5 conv     32       1 x 1/ 1    304 x 304 x  64 ->  304 x 304 x  32 0.379 BF
   6 conv     64       3 x 3/ 1    304 x 304 x  32 ->  304 x 304 x  64 3.407 BF
   7 Shortcut Layer: 4,  wt = 0, wn = 0, outputs: 304 x 304 x  64 0.006 BF
   8 conv     64       1 x 1/ 1    304 x 304 x  64 ->  304 x 304 x  64 0.757 BF
   9 route  8 2 	                           ->  304 x 304 x 128 
  10 conv     64       1 x 1/ 1    304 x 304